<a href="https://colab.research.google.com/github/bhargavreddy111/Homesite-Quote-Conversion/blob/master/Insurance%20Quote%20Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install vecstack

In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/Assignment3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/Assignment3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...    

In [0]:
#Copy Train data excluding target
trainData_Copy = train_data.iloc[:, :-1].copy()
testData_Copy = test_data.iloc[:, :-1].copy()

combined_Data = pd.concat([trainData_Copy,testData_Copy], keys=[0,1])

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)


#Select just Target Column
y_train = train_data.iloc[:, -1]


print(X_train.shape)
print(X_test.head()) 

print(y_train.shape)


(65000, 595)
   CoverageField11A  ...  GeographicField64_TX
0                13  ...                     0
1                 4  ...                     0
2                 3  ...                     0
3                 5  ...                     0
4                12  ...                     0

[5 rows x 595 columns]
(65000,)


In [0]:
#CONSTRUCT DEFAULT DECISION TREE on the imbalanced dataset==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)



In [0]:
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


=== All AUC Scores ===
[0.79929705 0.81587119 0.82037138 0.81261193 0.80698814 0.82484684
 0.82760855 0.80935996 0.81998486 0.78866754]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.8125607463146922


In [0]:
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')
print("=== Mean balanced Accuracy Score ===")
print("Mean balanced accuracy Score - Decision Tree: ",clf_cv_score.mean())

[0.7966409  0.81246981 0.8204948  0.81853081 0.80827783 0.82210648
 0.82837596 0.80320492 0.82580772 0.79232789]


=== Mean balanced Accuracy Score ===
Mean balanced accuracy Score - Decision Tree:  0.8128237106463393


In [0]:
#SMOTE with minority class at 100% of majority class
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=1)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 52738})


In [0]:
#Decision Tree model after SMOTE
clfsdt = DecisionTreeClassifier()
clfsdt.fit(X_res, y_res)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
clf_cv_score = cross_val_score(clfsdt, X_res, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

=== All AUC Scores ===
[0.38888889 0.673303   0.87144482 0.95240804 0.94823663 0.94416003
 0.931551   0.94444444 0.91778874 0.51090461]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.8083130203516011


In [0]:
#run cross-validation, get accuracy score
clf_cv_score = cross_val_score(clfsdt, X_res, y_res, cv=10, scoring="accuracy")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean accuracy Score - Decision Tree: ",clf_cv_score.mean())

[0.77777778 0.73331437 0.89827455 0.97193781 0.96852484 0.96786121
 0.97250664 0.96530148 0.96643277 0.64915608]


=== Mean Accuracy Score ===
Mean accuracy Score - Decision Tree:  0.8871087521586813


In [0]:
#SMOTE with minority class at 50% of majority class
sm1 = SMOTE(sampling_strategy='float', ratio=0.5)
print('Original dataset shape %s' % Counter(y_train))
X_res1, y_res1 = sm1.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res1))

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [0]:
clfsdt1 = DecisionTreeClassifier()
clfsdt1.fit(X_res1, y_res1)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
clf_cv_score = cross_val_score(clfsdt1, X_res1, y_res1, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

=== All AUC Scores ===
[0.48682215 0.65547971 0.70003792 0.67169132 0.80972696 0.95041714
 0.94946909 0.9051953  0.93163411 0.50436087]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7564834567673556


In [0]:
#run cross-validation, get accuracy score
clf_cv_score = cross_val_score(clfsdt1, X_res1, y_res1, cv=10, scoring="accuracy")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean accuracy Score - Decision Tree: ",clf_cv_score.mean())

[0.40487928 0.66565542 0.72190621 0.6935912  0.83364935 0.93667046
 0.93376311 0.88041967 0.91150442 0.34302693]


=== Mean Accuracy Score ===
Mean accuracy Score - Decision Tree:  0.7325066054643846


In [0]:
# SMOTE with minority class at 75% of majority class
sm2 = SMOTE(sampling_strategy='float', ratio=0.75)
X_res2, y_res2 = sm2.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res2))

Resampled dataset shape Counter({0: 52738, 1: 39553})


In [0]:
clfsdt2 = DecisionTreeClassifier()
clfsdt2.fit(X_res2, y_res2)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
clf_cv_score = cross_val_score(clfsdt2, X_res2, y_res2, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

=== All AUC Scores ===
[0.44010399 0.5369364  0.71722729 0.92845148 0.95215486 0.95762196
 0.95259731 0.93183499 0.91889518 0.51024099]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7846064450216772


In [0]:
#run cross-validation, get accuracy score
clf_cv_score = cross_val_score(clfsdt2, X_res2, y_res2, cv=10, scoring="accuracy")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean accuracy Score - Decision Tree: ",clf_cv_score.mean())

[0.40660888 0.5312026  0.74442037 0.92458555 0.94333081 0.95015711
 0.94603966 0.92144328 0.9083225  0.44061552]


=== Mean Accuracy Score ===
Mean accuracy Score - Decision Tree:  0.7716726265701819


In [0]:
#1st Model:Hyperparameter tuning done for decision tree classifier after SMOTE
#RANDOM SEARCH--------------------------------------------
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(15,90,10),'max_depth': range(10,40,5),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clfsdt,parameters,n_iter=15,cv=5)
clf_random.fit(X_res, y_res)
grid_parm=clf_random.best_params_
print(grid_parm)

In [0]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clfsdt = DecisionTreeClassifier(**grid_parm)

clfsdt.fit(X_res,y_res)
clfsdt_predict =pd.DataFrame(clfsdt.predict(X_test), columns=["QuoteConversion_Flag"])

clf_cv_score = cross_val_score(clfsdt, X_res, y_res, cv=10, scoring="roc_auc")
print(clf_cv_score)
print('\n')

In [0]:
#Result- Decision tree result after smote and hyper parameter tuning
X_Test_id=test_data["QuoteNumber"]
pd.concat([X_Test_id,clfsdt_predict],axis=1).to_csv("/gdrive/My Drive/Assignment3/Test_results1.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment3/Test_results1.csv')
res.head()

In [0]:
#2nd Model-Randomforest classifier with hyperparameter tuning after SMOTE
rand_parameters={'min_samples_leaf' : range(15,90,10),'max_depth': range(1,20,2),'max_features':[2,3,4],'n_estimators':[10,20,30]}
rfc = RandomForestClassifier()

#RANDOMIZED SEARCH----------------------------------------
rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=10,cv=5)
rfc_random.fit(X_res, y_res)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
rfc1= RandomForestClassifier(**grid_parm_rfc)
rfc1.fit(X_res,y_res)
rfc1_predict = pd.DataFrame(rfc1.predict(X_test),columns=["QuoteConversion_Flag"])


{'n_estimators': 20, 'min_samples_leaf': 25, 'max_features': 4, 'max_depth': 17}


In [0]:
pd.concat([X_Test_id,rfc1_predict],axis=1).to_csv("/gdrive/My Drive/Assignment3/Test_results2.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment3/Test_results2.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#3rd Model-KNN classifier with hyperparameter tuning after SMOTE
from sklearn.neighbors import KNeighborsClassifier
Knn_parameters={'n_neighbors':[5,10,15], 'leaf_size':[20,30,40]}
knnc= KNeighborsClassifier()

In [0]:
#RANDOMIZED SEARCH----------------------------------------
knnc_random = RandomizedSearchCV(knnc,Knn_parameters,n_iter=10,cv=5)
knnc_random.fit(X_res, y_res)
grid_parm_knnc=knnc_random.best_params_
print(grid_parm_knnc)


{'n_neighbors': 5, 'leaf_size': 20}


In [0]:
knnc1= KNeighborsClassifier(**grid_parm_knnc)
knnc1.fit(X_res,y_res)
knnc1_predict = pd.DataFrame(knnc1.predict(X_test),columns=["QuoteConversion_Flag"])

In [0]:
pd.concat([X_Test_id,knnc1_predict],axis=1).to_csv("/gdrive/My Drive/Assignment3/Test_results3.csv", index = None)

res3=pd.read_csv('/gdrive/My Drive/Assignment3/Test_results3.csv')
res3.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,1


In [0]:
#4th Model- MLP classifier after SMOTE
mlpc=MLPClassifier(hidden_layer_sizes =(50,10,100),activation='relu',max_iter=200)
mlpc.fit(X_res,y_res)
mlpc_predict =pd.DataFrame(mlpc.predict(X_test), columns=["QuoteConversion_Flag"])


print("accuracy Score (training) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))

mlpc_cv_score = cross_val_score(mlpc, X_res, y_res, cv=10, scoring="roc_auc")
print(mlpc_cv_score)
print('\n')


accuracy Score (training) after hypertuning randomized search for Decision Tree:1.000000
[0.32852233 0.5        0.49905195 0.50085324 0.51965639 0.5
 0.5        0.49971559 0.47041532 0.5       ]




In [0]:
pd.concat([X_Test_id,mlpc_predict],axis=1).to_csv("/gdrive/My Drive/Assignment3/Test_results4.csv", index = None)

res3=pd.read_csv('/gdrive/My Drive/Assignment3/Test_results4.csv')
res3.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#5th Model- SVM classifier after SMOTE
svmc=SVC(C=1.0, kernel='rbf', degree=3)
svmc.fit(X_res,y_res)
svmc_predict =pd.DataFrame(svmc.predict(X_test), columns=["QuoteConversion_Flag"])


print("accuracy Score (training) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))

svmc_cv_score = cross_val_score(svmc, X_res, y_res, cv=10, scoring="roc_auc")
print(svmc_cv_score)
print('\n')



In [0]:
pd.concat([X_Test_id,svmc_predict],axis=1).to_csv("/gdrive/My Drive/Assignment3/Test_results5.csv", index = None)

res3=pd.read_csv('/gdrive/My Drive/Assignment3/Test_results5.csv')
res3.head()

In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using KNN, MLP,RandomForest and Decision Tree Classifier\n")

models = [ DecisionTreeClassifier(), RandomForestClassifier(), KNeighborsClassifier(), MLPClassifier()  ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using KNN, MLP,RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [4]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.92366325]
    fold  1:  [0.92408039]
    fold  2:  [0.92331614]
    fold  3:  [0.92407464]
    ----
    MEAN:     [0.92378361] + [0.00031851]
    FULL:     [0.92378361]

model  1:     [RandomForestClassifier]
    fold  0:  [0.93090633]
    fold  1:  [0.93333333]
    fold  2:  [0.93366960]
    fold  3:  [0.93556584]
    ----
    MEAN:     [0.93336878] + [0.00165699]
    FULL:     [0.93336873]

model  2:     [KNeighborsClassifier]
    fold  0:  [0.72662116]
    fold  1:  [0.72810011]
    fold  2:  [0.72864836]
    fold  3:  [0.72443871]
    ----
    MEAN:     [0.72695209] + [0.00162956]
    FULL:     [0.72695210]

model  3:     [ML

In [0]:
# Decision Tree as secondary model in Stacking
model = DecisionTreeClassifier()
#Hyperparameter tuning done for decision tree classifier
#RANDOM SEARCH--------------------------------------------
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(15,90,10),'max_depth': range(10,40,5),'criterion':['gini','entropy']}
stack_random = RandomizedSearchCV(model,parameters,n_iter=15,cv=5)
stack_random.fit(S_Train, y_res)
grid_parm=stack_random.best_params_
print(grid_parm)
    

RandomizedSearchCV-Decision tree
{'min_samples_leaf': 65, 'max_depth': 25, 'criterion': 'entropy'}


In [0]:
stack_model = DecisionTreeClassifier(**grid_parm)


stack_model.fit(S_Train,y_res)
stack_predict=pd.DataFrame(stack_model.predict(S_Test), columns=["QuoteConversion_Flag"])

In [0]:
X_Test_id=test_data["QuoteNumber"]
pd.concat([X_Test_id,stack_predict],axis=1).to_csv("/gdrive/My Drive/Assignment3/Test_results6.csv", index = None)

res4=pd.read_csv('/gdrive/My Drive/Assignment3/Test_results6.csv')
res4.head()


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0
